In [ ]:
import pandas as pd
import numpy as np
import math
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import warnings, sys
from xgboost.sklearn import XGBClassifier
from sklearn.svm import SVC
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
df = pd.read_csv(r'./Data/overall_20210531.csv')
df2 = pd.read_csv(r'./Data/overall_20210524.csv')
matrix = pd.read_csv(r'./Data/matrix_all.csv')
#selected columns by AR
ar_col = ['稅前淨利', '歸屬母公司淨利（損）', '常續性稅後淨利', '合併總損益','稅前淨利－CFO', '繼續營業單位損益','資產總額', '流動資產', '母公司股東權益合計','股東權益總額', '負債及股東權益總額']
#selected columns by XGB
xgb_col = ['季','稅前淨利成長率','不重分類至損益之項目－OCI','經常淨利成長率']
print(df.shape,df2.shape)

(18496, 130) (18496, 180)


In [ ]:
print(df.shape,df2.shape)
df2 = df2[['代號','名稱','年份','季','開盤價','收盤價']]
df3 = pd.merge(df,df2,how='left',on=['代號','名稱','年份','季'])
print(df3.shape)
df3.head()

(18496, 132)


,代號,名稱,年份,季,營收成長率,營業毛利成長率,已實現銷貨毛利成長率,營業利益成長率,稅前淨利成長率,稅後淨利成長率,經常淨利成長率,常續淨利成長率,總資產成長率,淨值成長率,折舊性FA成長率,總資產報酬成長率,營收變動率,營業利益變動率,淨利變動率（單季）,稅前盈餘變動率,營業收入淨額,營業成本,營業毛利,已實現銷貨毛利,營業費用,研究發展費,營業利益,利息收入,其他收入,其他利益及損失,財務成本,採權益法之關聯企業及合資損益之份額,營業外收入及支出,稅前淨利,所得稅費用,繼續營業單位損益,合併總損益,不重分類至損益之項目－OCI,後續可能重分類至損益之項目－OCI,其他綜合損益－OCI,...,其他非流動資產,非流動資產,資產總額,短期借款,透過損益按公允價值衡量之金融負債－流動,合約負債－流動,應付帳款及票據,其他應付款,當期所得稅負債,負債準備－流動,一年內到期長期負債,流動負債,銀行借款－非流動,應計退休金負債,遞延所得稅,非流動負債,負債總額,普通股股本,股本,資本公積合計,法定盈餘公積,特別盈餘公積,未分配盈餘,保留盈餘,其他權益,庫藏股票帳面值,母公司股東權益合計,非控制權益,股東權益總額,負債及股東權益總額,單季營收(千元),單季營收成長,來自營運之現金流量/營業收入淨額,來自營運之現金流量/ 繼續營業單位損益,來自營運現金流量成長率,應收帳款週轉次數,存貨週轉率（次）,進場訊號,開盤價,收盤價
0,1471,首利,2020,4,3.55,873.06,873.06,45.97,254.92,250.96,250.96,43.24,26.00,60.83,-46.1019,7.01,-43.91,-325.25,-79.91,-78.89,241949,212333,29616,29616,53461,2314,-23845,185,8795,107566,2005,0,114356,90511,7372,83139,83139,132,-4415,-4283,...,55447,442012,1595001,80000,0,0,202854,98639,26,129071,18657,536061,10883,0,0,15573,551634,1504145,1504145,3539,0,0,-534665,-534665,70388,0,1043407,-40,1043367,1595001,241949,0.377110,-0.109416,-0.318419,59.047383,0.80,2.41,1,6.0,7.30
1,1503,士電,2020,4,10.71,-6.45,-6.45,-13.94,-22.23,-39.65,-39.65,-40.02,3.19,4.16,-0.0697,-0.36,20.88,-45.16,-46.51,-46.98,7262739,6138053,1124686,1124686,845686,166352,279000,9288,10382,5390,11440,33122,37454,316454,85457,230997,230997,198889,69897,268786,...,7928401,24755090,42057817,2145977,10419,1663954,5848635,1450559,264055,973862,0,12658049,850000,407380,2263326,3699423,16357472,5209722,5209722,2626221,2730986,5136954,8750888,16618828,717532,0,25172303,528042,25700345,42057817,7262361,0.388073,0.105486,3.316558,117.784897,1.24,1.36,0,50.6,49.20
2,1504,東元,2020,4,3.44,3.16,3.16,30.24,21.68,37.02,37.02,30.38,6.64,9.59,-4.9576,0.30,8.97,21.07,15.30,4.05,12474622,9910318,2564304,2564629,1703907,247603,860722,31038,135040,19954,47791,36318,143521,1004243,104373,899870,899870,-745408,184386,-561022,...,10916535,59886719,105679071,2816832,0,1493931,8018643,4902975,490113,406874,302331,19618968,3309400,1656948,2539962,18434871,38053839,19676929,19676929,7386901,7024635,3640779,17271503,27936917,7339258,-511710,61828295,5796937,67625232,105679071,12385855,0.356141,0.190255,2.637443,4.224476,1.25,1.01,1,27.7,32.35
3,1506,正道,2020,4,-13.63,184.84,184.84,59.16,74.51,73.30,73.30,62.34,-16.10,2.37,-17.2768,4.81,5.88,25.37,22.67,22.51,211845,198022,13823,13823,54570,7977,-40747,103,-832,-3770,3299,3720,-4181,-44928,-20,-44908,-44908,2442,4307,6749,...,11972,1312219,2289883,454606,0,0,109720,184011,0,0,77782,865931,384896,15381,100891,521112,1387043,1631618,1631618,139516,0,0,-793236,-793236,-124261,0,853637,49203,902840,2289883,213851,0.031696,0.035989,-0.169769,-85.557050,1.09,0.47,0,10.8,9.75
4,1507,永大,2020,4,20.18,20.96,20.96,13.12,34.52,55.61,55.61,37.14,10.38,5.40,-2.4472,0.50,6.10,-20.61,-24.82,-36.06,4492021,3433267,1058754,1058754,758439,129078,300315,14744,17090,43946,236,4578,65378,365693,25275,340418,340418,-158423,145462,-12961,...,1275216,7471896,23916553,0,0,6452490,3723544,939780,159658,0,0,11327860,0,259929,3666,546393,11874253,4108200,4108200,279398,3171035,307639,4117021,7595695,-79854,-69411,11834028,208272,12042300,23916553,4472170,0.005277,0.081834,1.079852,-10.728787,1.07,0.59,0,62.7,60.00


## Preprocessing

In [ ]:
df = pd.read_csv(r'./Data/overall_20210524.csv')
# Drop Unused Columns
df.drop(['最高價', '最低價', '成交量(股)', '交易額(萬)', '開盤價', '收盤價', '漲跌幅(%)', '年/月'], 1, inplace=True)
#(18496,172)

# Sort Columns
first = ['代號', '名稱', '年份', '季']
change_cols = first + [c for c in df.columns if c not in first + ['進場訊號']] + ['進場訊號']
df = df[change_cols]
df.replace({np.inf: 0}, inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
# Drop columns that contains too much NaN
c = 0
na_col = []
for x in df.columns[5:-1]:
    zero = len(df[df[x]==0])
    if round(zero/18496, 2) >= 0.8:
        na_col.append(x)
        c += 1
print(na_col)
df.drop(na_col, axis = 1, inplace = True)
#(18496,130)

['聯屬公司已（未）實現銷貨利益', '預期信用減損（損失）利益－營業費用', '其他收益及費損淨額', '預期信用減損（損失）利益', '除列按攤銷後成本衡量金融資產淨損益', '金融資產重分類淨損益', '營業外收入及支出－其他', '停業單位損益', '其他損益調整項－非常項目及累計影響數', '發放特別股股息', '新增投資－CFI', '出售投資－CFI', '現金增（減）資－CFF', '避險之金融資產－流動', '合約資產－流動', '資金貸予他人－流動', '待出售非流動資產', '按攤銷後成本衡量之金融資產－非流動', '避險之金融資產－非流動', '合約資產－非流動', '預付投資款', '使用權資產', '應付商業本票∕承兌匯票', '避險之金融負債－流動', '按攤銷後成本衡量之金融負債－流動', '與待出售非流動資產直接相關之負債', '租賃負債─流動', '特別股負債－流動', '透過損益按公允價值衡量之金融負債－非流動', '避險之金融負債－非流動', '按攤銷後成本衡量之金融負債－非流動', '合約負債－非流動', '特別股負債－非流動', '應付公司債－非流動', '其他長期借款－非流動', '租賃負債－非流動', '負債準備－非流動', '遞延貸項', '特別股股本', '預收股款', '待分配股票股利', '換股權利證書']


In [ ]:
#seperate the data into signal 1 and 0
df_1 = df[df['進場訊號'] == 1]
df_0 = df[df['進場訊號'] == 0]
print(df_1.shape,df_0.shape)

(4785, 130) (13711, 130)


In [ ]:
direct = pd.read_excel(r'./Data/方向_2.xlsx')
direct['remove'] = direct['columns'].apply(lambda x: 1 if x in na_col else 0)
direct = direct[direct['remove'] == 0].drop('remove', axis = 1)

In [ ]:
#將不同特性的欄位分隔出來處理
import math
to_remove = []
dir_s = direct[direct['label'] == 's']['columns'].to_list()
dir_big = direct[direct['label'] == '+']['columns'].to_list()
dir_small = direct[direct['label'] == '-']['columns'].to_list()
#split the dataframe
df_s = df[dir_s]
df_big = df[dir_big]
df_small = df[dir_small]
df_1_s = df_1[dir_s]
df_1_big = df_1[dir_big]
df_1_small = df_1[dir_small]
df_0_s = df_0[dir_s]
df_0_big = df_0[dir_big]
df_0_small = df_0[dir_small]

In [ ]:
#處理正值越高，負值越小的欄位
def xxx(d, a, b):
    if d < 0 and d > b:
        return 1
    elif d > a:
        return 1
    else:
        return 0
for i in df_s.columns:
  if min(df_s[i]) < 0:
    a = [x for x in df_s[i] if x >= 0]
    b = [x for x in df_s[i] if x < 0]
    a75 = np.percentile(a, 75)
    b75 = np.percentile(b, 75)
    df_s[i] = df_s[i].apply(lambda y : xxx(y, a75, b75))
  else:
    a75 = np.percentile(dd[x], 75)
    df_s[i] = df_s[i].apply(lambda y:1 if y > a75 else 0)
#df_1_s
for i in df_s.columns:
  if min(df_1_s[i]) < 0:
    a = [x for x in df_1_s[i] if x >= 0]
    b = [x for x in df_1_s[i] if x < 0]
    a75 = np.percentile(a, 75)
    b75 = np.percentile(b, 75)
    df_1_s[i] = df_1_s[i].apply(lambda y : xxx(y, a75, b75))
  else:
    a75 = np.percentile(dd[x], 75)
    df_1_s[i] = df_1_s[i].apply(lambda y:1 if y > a75 else 0)
#df_0_s
for i in df_0_s.columns:
  if min(df_0_s[i]) < 0:
    a = [x for x in df_0_s[i] if x >= 0]
    b = [x for x in df_0_s[i] if x < 0]
    a75 = np.percentile(a, 75)
    b75 = np.percentile(b, 75)
    df_0_s[i] = df_0_s[i].apply(lambda y : xxx(y, a75, b75))
  else:
    a75 = np.percentile(dd[x], 75)
    df_0_s[i] = df_0_s[i].apply(lambda y:1 if y > a75 else 0)

In [ ]:
#處理數值越大越好的欄位
for i in df_big.columns:
  df_big[i] = df_big[i].apply(lambda x: 1 if x > np.percentile(df_big[i], 75) else 0)
for i in df_1_big.columns:
  df_1_big[i] = df_1_big[i].apply(lambda x: 1 if x > np.percentile(df_1_big[i], 75) else 0)
for i in df_0_big.columns:
  df_0_big[i] = df_0_big[i].apply(lambda x: 1 if x > np.percentile(df_0_big[i], 75) else 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [ ]:
#處理數值越小越好的欄位
for i in df_small.columns:
  df_small[i] = df_small[i].apply(lambda x: 1 if x < np.percentile(df_small[i], 25) else 0)
for i in df_1_small.columns:
  df_1_small[i] = df_1_small[i].apply(lambda x: 1 if x < np.percentile(df_1_small[i], 25) else 0)
for i in df_0_small.columns:
  df_0_small[i] = df_0_small[i].apply(lambda x: 1 if x < np.percentile(df_0_small[i], 25) else 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [ ]:
#將處理完資料整理成關聯規則需要的稀疏矩陣
matrix = pd.concat([df_s, df_big, df_small] ,axis = 1)
matrix_1 = pd.concat([df_1_s,df_1_big,df_1_small], axis = 1)
matrix_0 = pd.concat([df_0_s,df_0_big,df_0_small], axis = 1)

## 函式

In [ ]:
#Function
#find dimention with SMOTE
def find_dimention(feature, tar, dimention, model):
  score = []
  com = []
  X_res, Y_res = SMOTE(random_state = 42).fit_resample(feature, tar)
  for i in dimention:
    std = StandardScaler()
    pca = PCA(n_components=i)
    std_x = std.fit_transform(X_res)
    pca_x = pca.fit_transform(std_x)
    X_train, X_test, y_train, y_test = train_test_split(pca_x, Y_res, test_size = 0.2, random_state = 42, stratify = Y_res)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)
    roc = roc_auc_score(y_test, pred)
    com.append(i)
    score.append(roc)
  zipped = list(zip(com,score))
  temp = sorted(zipped, key = lambda x: x[1])
  top3 = temp[-3:]
  for i,j in top3:
    std = StandardScaler()
    pca = PCA(n_components=i)
    std_x = std.fit_transform(X_res)
    pca_x = pca.fit_transform(std_x)
    X_train, X_test, y_train, y_test = train_test_split(pca_x, Y_res, test_size = 0.2, random_state = 42, stratify = Y_res)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    pred_train = model.predict(X_train)
    var = pca.explained_variance_[0:6]
    percent = pca.explained_variance_ratio_[0:6]
    acc = accuracy_score(y_test, pred)
    roc = roc_auc_score(y_test, pred)
    roc_train = roc_auc_score(y_train, pred_train)
    cf = confusion_matrix(y_test, pred)
    print('/PCA/')
    print('Dimension:{}'.format(i))
    print('Varience:{}'.format(var))
    print('Ratio:',percent)
    print('/Model/')
    print('ROC_train:{}'.format(roc_train))
    print('ACC:{}'.format(acc))
    print('ROC:{}'.format(roc))
    print('Confusion Matrix:\n{}\n'.format(cf))
    print('==================================================')

#Train model and tuning with SMOTE
def pca_random_model(feature, tar, dimention, model, para):
  reg = []
  reg.append(model)
  X_res, Y_res = SMOTE(random_state = 42).fit_resample(feature, tar)
  std = StandardScaler()
  pca = PCA(n_components=dimention)
  std_x = std.fit_transform(X_res)
  pca_x = pca.fit_transform(std_x)
  X_train, X_test, y_train, y_test = train_test_split(pca_x, Y_res, test_size = 0.2, random_state = 42, stratify = Y_res)
  grid = RandomizedSearchCV(model, param_distributions=para, scoring = 'roc_auc',n_iter=10)
  grid.fit(X_train, y_train)
  print(grid.best_params_)
  model = grid.best_estimator_
  model.fit(X_train, y_train)
  pred = model.predict(X_test)
  pred_train = model.predict(X_train)
  var = pca.explained_variance_[0:6]
  percent = pca.explained_variance_ratio_[0:6]
  acc = accuracy_score(y_test, pred)
  roc = roc_auc_score(y_test, pred)
  cf = confusion_matrix(y_test, pred)
  roc_train = roc_auc_score(y_train, pred_train)
  print('/PCA/')
  print('Dimension:{}'.format(dimention))
  print('Varience:{}'.format(var))
  print('Ratio:',percent)
  print('/Model/')
  print('ROC_train:{}'.format(roc_train))
  print('ACC:{}'.format(acc))
  print('ROC:{}'.format(roc))
  print('Confusion Matrix:\n{}\n'.format(cf))

#usual training with smote
def normal_training_smote(feature, tar, dimention, model):
  X_res, Y_res = SMOTE(random_state = 42).fit_resample(feature, tar)
  std = StandardScaler()
  pca = PCA(n_components=dimention)
  std_x = std.fit_transform(X_res)
  pca_x = pca.fit_transform(std_x)
  X_train, X_test, y_train, y_test = train_test_split(pca_x, Y_res, test_size = 0.2, random_state = 42, stratify = Y_res)
  model.fit(X_train, y_train)
  pred_train = model.predict(X_train)
  pred = model.predict(X_test)
  var = pca.explained_variance_[0:6]
  percent = pca.explained_variance_ratio_[0:6]
  acc = accuracy_score(y_test, pred)
  roc = roc_auc_score(y_test, pred)
  roc_train = roc_auc_score(y_train, pred_train)
  cf = confusion_matrix(y_test, pred)
  print('/PCA/')
  print('Dimension:100')
  print('Varience:{}'.format(var))
  print('Ratio:',percent)
  print('/Model/')
  print('ROC_train:{}'.format(roc_train))
  print('ACC:{}'.format(acc))
  print('ROC:{}'.format(roc))
  print('Confusion Matrix:\n{}\n'.format(cf))
  print('==================================================')

def normal_training_pca(data,feature, tar, dimention, model):
  std = StandardScaler()
  pca = PCA(n_components=dimention)
  std_x = std.fit_transform(X)
  pca_x = pca.fit_transform(std_x)
  X_train, X_test, y_train, y_test = train_test_split(pca_x, y, test_size = 0.2, random_state = 42, stratify = y)
  model.fit(X_train, y_train)
  pred_train = model.predict(X_train)
  pred = model.predict(X_test)
  var = pca.explained_variance_[0:6]
  percent = pca.explained_variance_ratio_[0:6]
  acc = accuracy_score(y_test, pred)
  roc = roc_auc_score(y_test, pred)
  roc_train = roc_auc_score(y_train, pred_train)
  cf = confusion_matrix(y_test, pred)
  print('/PCA/')
  print('Dimension:{}'.format(dimention))
  print('Varience:{}'.format(var))
  print('Ratio:',percent)
  print('/Model/')
  print('ROC_train:{}'.format(roc_train))
  print('ACC:{}'.format(acc))
  print('ROC:{}'.format(roc))
  print('Confusion Matrix:\n{}\n'.format(cf))
  print('==================================================')
  a = pd.DataFrame({'進場訊號':y_test,'預測進場訊號':pred})
  b = pd.DataFrame({'進場訊號':y_train,'預測進場訊號':pred_train})
  c = pd.concat([a,b],axis = 0)
  d = data[['代號','名稱','年份','季','開盤價','收盤價']]
  e = pd.concat([d,c],axis = 1)
  e = e.sort_values(by = ['代號','名稱','年份','季'])
  return e




## 關聯規則欄位篩選後預測

In [ ]:
apr_1 = apriori(matrix_1, min_support=0.2,use_colnames=True)
ar_1 = association_rules(apr_1)
apr_0 = apriori(matrix_0, min_support=0.2,use_colnames=True)
ar_0 = association_rules(apr_0)
apr_all = apriori(matrix, min_support=0.2,use_colnames=True)
ar_all = association_rules(apr_all)
arlist = [ar_0,ar_all,ar_all]
for i in arlist:
  i['length1'] = i['antecedents'].apply(lambda x: len(x))
  i['length2'] = i['consequents'].apply(lambda x: len(x))
  i['length'] = i['length1']+i['length2'] 
  i.sort_values(by = ['length','confidence','lift'], ascending=False, inplace=True)
  i.drop(['length1','length2'], axis = 1,inplace = True)
#透過進場訊號為0,1以及全部的三個資料觀察關聯規則中重要的欄位有哪些
ar_col = ['稅前淨利', '歸屬母公司淨利（損）', '常續性稅後淨利', '合併總損益','稅前淨利－CFO', '繼續營業單位損益','資產總額', '流動資產', '母公司股東權益合計','股東權益總額', '負債及股東權益總額']

In [ ]:
#將需要的資料欄位處理好
df = pd.read_csv(r'./Data/overall_20210531.csv')
df2 = pd.read_csv(r'./Data/overall_20210524.csv')
print(df.shape,df2.shape)
df2 = df2[['代號','名稱','年份','季','開盤價','收盤價']]
df3 = pd.merge(df,df2,how='left',on=['代號','名稱','年份','季'])
print(df3.shape)
df3.head()

(18496, 130) (18496, 180)
(18496, 132)


,代號,名稱,年份,季,營收成長率,營業毛利成長率,已實現銷貨毛利成長率,營業利益成長率,稅前淨利成長率,稅後淨利成長率,經常淨利成長率,常續淨利成長率,總資產成長率,淨值成長率,折舊性FA成長率,總資產報酬成長率,營收變動率,營業利益變動率,淨利變動率（單季）,稅前盈餘變動率,營業收入淨額,營業成本,營業毛利,已實現銷貨毛利,營業費用,研究發展費,營業利益,利息收入,其他收入,其他利益及損失,財務成本,採權益法之關聯企業及合資損益之份額,營業外收入及支出,稅前淨利,所得稅費用,繼續營業單位損益,合併總損益,不重分類至損益之項目－OCI,後續可能重分類至損益之項目－OCI,其他綜合損益－OCI,...,其他非流動資產,非流動資產,資產總額,短期借款,透過損益按公允價值衡量之金融負債－流動,合約負債－流動,應付帳款及票據,其他應付款,當期所得稅負債,負債準備－流動,一年內到期長期負債,流動負債,銀行借款－非流動,應計退休金負債,遞延所得稅,非流動負債,負債總額,普通股股本,股本,資本公積合計,法定盈餘公積,特別盈餘公積,未分配盈餘,保留盈餘,其他權益,庫藏股票帳面值,母公司股東權益合計,非控制權益,股東權益總額,負債及股東權益總額,單季營收(千元),單季營收成長,來自營運之現金流量/營業收入淨額,來自營運之現金流量/ 繼續營業單位損益,來自營運現金流量成長率,應收帳款週轉次數,存貨週轉率（次）,進場訊號,開盤價,收盤價
0,1471,首利,2020,4,3.55,873.06,873.06,45.97,254.92,250.96,250.96,43.24,26.00,60.83,-46.1019,7.01,-43.91,-325.25,-79.91,-78.89,241949,212333,29616,29616,53461,2314,-23845,185,8795,107566,2005,0,114356,90511,7372,83139,83139,132,-4415,-4283,...,55447,442012,1595001,80000,0,0,202854,98639,26,129071,18657,536061,10883,0,0,15573,551634,1504145,1504145,3539,0,0,-534665,-534665,70388,0,1043407,-40,1043367,1595001,241949,0.377110,-0.109416,-0.318419,59.047383,0.80,2.41,1,6.0,7.30
1,1503,士電,2020,4,10.71,-6.45,-6.45,-13.94,-22.23,-39.65,-39.65,-40.02,3.19,4.16,-0.0697,-0.36,20.88,-45.16,-46.51,-46.98,7262739,6138053,1124686,1124686,845686,166352,279000,9288,10382,5390,11440,33122,37454,316454,85457,230997,230997,198889,69897,268786,...,7928401,24755090,42057817,2145977,10419,1663954,5848635,1450559,264055,973862,0,12658049,850000,407380,2263326,3699423,16357472,5209722,5209722,2626221,2730986,5136954,8750888,16618828,717532,0,25172303,528042,25700345,42057817,7262361,0.388073,0.105486,3.316558,117.784897,1.24,1.36,0,50.6,49.20
2,1504,東元,2020,4,3.44,3.16,3.16,30.24,21.68,37.02,37.02,30.38,6.64,9.59,-4.9576,0.30,8.97,21.07,15.30,4.05,12474622,9910318,2564304,2564629,1703907,247603,860722,31038,135040,19954,47791,36318,143521,1004243,104373,899870,899870,-745408,184386,-561022,...,10916535,59886719,105679071,2816832,0,1493931,8018643,4902975,490113,406874,302331,19618968,3309400,1656948,2539962,18434871,38053839,19676929,19676929,7386901,7024635,3640779,17271503,27936917,7339258,-511710,61828295,5796937,67625232,105679071,12385855,0.356141,0.190255,2.637443,4.224476,1.25,1.01,1,27.7,32.35
3,1506,正道,2020,4,-13.63,184.84,184.84,59.16,74.51,73.30,73.30,62.34,-16.10,2.37,-17.2768,4.81,5.88,25.37,22.67,22.51,211845,198022,13823,13823,54570,7977,-40747,103,-832,-3770,3299,3720,-4181,-44928,-20,-44908,-44908,2442,4307,6749,...,11972,1312219,2289883,454606,0,0,109720,184011,0,0,77782,865931,384896,15381,100891,521112,1387043,1631618,1631618,139516,0,0,-793236,-793236,-124261,0,853637,49203,902840,2289883,213851,0.031696,0.035989,-0.169769,-85.557050,1.09,0.47,0,10.8,9.75
4,1507,永大,2020,4,20.18,20.96,20.96,13.12,34.52,55.61,55.61,37.14,10.38,5.40,-2.4472,0.50,6.10,-20.61,-24.82,-36.06,4492021,3433267,1058754,1058754,758439,129078,300315,14744,17090,43946,236,4578,65378,365693,25275,340418,340418,-158423,145462,-12961,...,1275216,7471896,23916553,0,0,6452490,3723544,939780,159658,0,0,11327860,0,259929,3666,546393,11874253,4108200,4108200,279398,3171035,307639,4117021,7595695,-79854,-69411,11834028,208272,12042300,23916553,4472170,0.005277,0.081834,1.079852,-10.728787,1.07,0.59,0,62.7,60.00


In [ ]:
#先找降到幾個維度比較好
y = df3.loc[:,'進場訊號']
X = df3[ar_col]
dim = [i for i in range(1,12)]
xgb = XGBClassifier()
find_dimention(X, y, dim, xgb)
#dimention = 9

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


/PCA/
Dimension:10
Varience:[9.76321934e+00 1.08878312e+00 1.16219807e-01 1.52894488e-02
 1.18467536e-02 3.03881991e-03]
Ratio: [8.87533027e-01 9.89766741e-02 1.05650517e-02 1.38989920e-03
 1.07693833e-03 2.76246281e-04]
/Model/
ROC_train:0.606963833292139
ACC:0.5795806745670009
ROC:0.5795867100740217
Confusion Matrix:
[[1499 1244]
 [1062 1680]]

/PCA/
Dimension:9
Varience:[9.76321934e+00 1.08878312e+00 1.16219807e-01 1.52894488e-02
 1.18467536e-02 3.03881991e-03]
Ratio: [8.87533027e-01 9.89766741e-02 1.05650517e-02 1.38989920e-03
 1.07693833e-03 2.76246281e-04]
/Model/
ROC_train:0.6073282604533871
ACC:0.5806745670009116
ROC:0.5806812673224571
Confusion Matrix:
[[1492 1251]
 [1049 1693]]

/PCA/
Dimension:8
Varience:[9.76321934e+00 1.08878312e+00 1.16219807e-01 1.52894488e-02
 1.18467536e-02 3.03881991e-03]
Ratio: [8.87533027e-01 9.89766741e-02 1.05650517e-02 1.38989920e-03
 1.07693833e-03 2.76246281e-04]
/Model/
ROC_train:0.6077844271559283
ACC:0.5810391978122151
ROC:0.5810448345008168

In [ ]:
# Tuning max_depth, min_child_weight
y = df3.loc[:,'進場訊號']
X = df3[sel_col]
dim = 9
xgb = XGBClassifier()
params = {'max_depth':range(1,15,2), 'min_child_weight':range(1,9,1)}
pca_random_model(X,y,dim,xgb,params)
#min_child_weight': 2, 'max_depth': 13

In [ ]:
# Tuning subsample,colsample_bytree
y = df3.loc[:,'進場訊號']
X = df3[sel_col]
dim = 9
xgb = XGBClassifier(min_child_weight=2, max_depth=13,eval_metric = 'auc')
params = {'subsample':[0.6,0.7,0.8,0.9,1], 'colsample_bytree':[0.6,0.7,0.8,0.9,1]}
pca_random_model(X,y,dim,xgb,params)
#'subsample': 0.9, 'colsample_bytree': 1

In [ ]:
#Tuning gamma
y = df3.loc[:,'進場訊號']
X = df3[sel_col]
dim = 9
xgb = XGBClassifier(min_child_weight=2, max_depth=13,eval_metric = 'auc',subsample=0.9, gamma = 0.4)
normal_training_smote(X,y,dim,xgb)

In [ ]:
# AR-BEST
y = df3.loc[:,'進場訊號']
X = df3[ar_col]
dim = 9
xgb = XGBClassifier(min_child_weight=2, max_depth=13,eval_metric = 'auc',subsample=0.9, gamma = 0.4, scale_pos_weight=10)
final_1 = normal_training_pca(df3,X,y,9,xgb)
final_1.to_csv('./Data/Method_B_Final_AR.csv', index=0, header=True, encoding='utf-8')

/PCA/
Dimension:9
Varience:[9.64854235e+00 1.18030128e+00 1.29642965e-01 2.47025717e-02
 1.12149984e-02 3.76160973e-03]
Ratio: [8.77092791e-01 1.07294316e-01 1.17850869e-02 2.24556692e-03
 1.01949019e-03 3.41946032e-04]
/Model/
ROC_train:0.937454412837345
ACC:0.5305405405405406
ROC:0.5425397449420982
Confusion Matrix:
[[1420 1323]
 [ 414  543]]



,代號,名稱,年份,季,開盤價,收盤價,進場訊號,預測進場訊號
18138,1471,首利,2010,1,34.20,23.75,0,0
17779,1471,首利,2010,2,23.85,24.50,0,0
17412,1471,首利,2010,3,24.50,20.10,0,0
17046,1471,首利,2010,4,20.20,17.15,0,0
16668,1471,首利,2011,1,17.15,13.90,0,0
...,...,...,...,...,...,...,...,...
2344,9912,偉聯,2019,4,8.44,5.80,0,1
1877,9912,偉聯,2020,1,5.76,6.82,1,1
1411,9912,偉聯,2020,2,6.77,7.54,1,1
943,9912,偉聯,2020,3,7.54,8.39,1,1


## 純PCA降維後預測

In [ ]:
# find XGB dim range
y = df3['進場訊號']
X = df3.loc[:,'季':'存貨週轉率（次）']
dim = [i for i in range(10, 130, 10)]
xgb = XGBClassifier()  
find_dimention(X, y, dim, xgb)
#dimention should be 100

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


/PCA/
Dimension:110
Varience:[44.97471976 10.51159282  5.79896221  4.21957978  3.641896    3.18937768]
Ratio: [0.3569292  0.0834223  0.04602183 0.03348751 0.02890288 0.0253116 ]
/Model/
ROC_train:0.7298639104540952
ACC:0.6964448495897904
ROC:0.6964418147592986
Confusion Matrix:
[[1956  787]
 [ 878 1864]]

/PCA/
Dimension:100
Varience:[44.97471976 10.51159282  5.79896221  4.21957978  3.641896    3.18937768]
Ratio: [0.3569292  0.0834223  0.04602183 0.03348751 0.02890288 0.0253116 ]
/Model/
ROC_train:0.7290432874983068
ACC:0.6979033728350046
ROC:0.6979002050973594
Confusion Matrix:
[[1962  781]
 [ 876 1866]]

/PCA/
Dimension:50
Varience:[44.97471976 10.51159282  5.79896221  4.21957978  3.641896    3.18937768]
Ratio: [0.3569292  0.0834223  0.04602183 0.03348751 0.02890288 0.0253116 ]
/Model/
ROC_train:0.7198356656139685
ACC:0.695897903372835
ROC:0.6958931733398429
Confusion Matrix:
[[1980  763]
 [ 905 1837]]



In [ ]:
#Tuning max_depth, min_child_weight
y = df3['進場訊號']
X = df3.loc[:,'季':'存貨週轉率（次）']
dim = 100
xgb = XGBClassifier()
params = {'max_depth':range(1,15,2), 'min_child_weight':range(1,9,1)}
pca_random_model(X,y,dim,xgb,params)
#max_depth = 13, min_child_weight = 5

In [ ]:
#Tuning gamma
y = df3['進場訊號']
X = df3.loc[:,'季':'存貨週轉率（次）']
dim = 100
xgb = XGBClassifier(n_estimators = 1000,max_depth=13, min_child_weight=5
                    ,eval_metric = 'auc', colsample_bytree = 0.8, subsample = 0.8,
                   gamma = 0.2)
normal_training_smote(X, y, dim, xgb)

In [ ]:
y = df3['進場訊號']
X = df3.loc[:,'季':'存貨週轉率（次）']
dim = 100
xgb = XGBClassifier(n_estimators = 1000,max_depth=13, min_child_weight=5
                    ,eval_metric = 'auc', colsample_bytree = 0.8, subsample = 0.8,
                   gamma = 0.2, scale_pos_weight = 10)
final_3 = normal_training_pca(df3,X, y, dim, xgb)
final_3.to_csv('./Data/Method_B_Final.csv', index=0, header=True, encoding='utf-8')

/PCA/
Dimension:100
Varience:[44.95134567 10.18376748  5.72809252  4.16369749  3.51999011  3.20836334]
Ratio: [0.35673742 0.08081918 0.04545859 0.03304343 0.02793492 0.02546182]
/Model/
ROC_train:1.0
ACC:0.7148648648648649
ROC:0.574666358863123
Confusion Matrix:
[[2373  370]
 [ 685  272]]



,代號,名稱,年份,季,開盤價,收盤價,進場訊號,預測進場訊號
18138,1471,首利,2010,1,34.20,23.75,0,0
17779,1471,首利,2010,2,23.85,24.50,0,0
17412,1471,首利,2010,3,24.50,20.10,0,0
17046,1471,首利,2010,4,20.20,17.15,0,0
16668,1471,首利,2011,1,17.15,13.90,0,0
...,...,...,...,...,...,...,...,...
2344,9912,偉聯,2019,4,8.44,5.80,0,1
1877,9912,偉聯,2020,1,5.76,6.82,1,1
1411,9912,偉聯,2020,2,6.77,7.54,1,1
943,9912,偉聯,2020,3,7.54,8.39,1,1


## XGBoost欄位篩選後預測

In [ ]:
#XGBoost after tuning(SMOTE)
y = df.loc[:,'進場訊號']
X = df.loc[:,'季': '存貨週轉率（次）']
X_res, Y_res = SMOTE(random_state = 42).fit_resample(X, y)
std = StandardScaler()
xgb = XGBClassifier(n_estimator = 1000,learning_rate = 0.1,max_depth = 9, min_child_weight = 5, subsample = 0.8, colsample_bytree = 0.8)
std_x = std.fit_transform(X_res)
X_train, X_test, y_train, y_test = train_test_split(std_x, Y_res, test_size = 0.2, random_state = 42, stratify = Y_res)
xgb.fit(X_train, y_train)
acc_train = accuracy_score(y_train, xgb.predict(X_train))
acc = accuracy_score(y_test, xgb.predict(X_test))
roc_train = roc_auc_score(y_train, xgb.predict(X_train))
roc_test = roc_auc_score(y_test, xgb.predict(X_test))
cf = confusion_matrix(y_test, xgb.predict(X_test))
imp = pd.DataFrame(columns =['column','importance'])
imp['column'] = X.columns
imp['importance'] = xgb.feature_importances_.round(4)
imp = imp.sort_values(by = 'importance', ascending = False)
print('ACC_train:{0:.3}'.format(acc_train))
print('ACC_test:{0:.3}'.format(acc))
print('ROC_train{0:.3}'.format(roc_train))
print('ROC_test{0:.3}'.format(roc_test))
print(cf)
imp.head(20)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


ACC_train:0.984
ACC_test:0.835
ROC_train0.984
ROC_test0.835
[[2444  299]
 [ 604 2138]]


,column,importance
0,季,0.1343
5,稅前淨利成長率,0.0381
34,不重分類至損益之項目－OCI,0.0245
8,常續淨利成長率,0.0141
66,支付現金股利－CFF,0.0107
118,負債及股東權益總額,0.0095
35,後續可能重分類至損益之項目－OCI,0.0088
115,母公司股東權益合計,0.0086
79,當期所得稅資產－流動,0.0085
3,已實現銷貨毛利成長率,0.0084


In [ ]:
#XGBoost after importance mask
sel_imp = imp[imp['importance'] >= 0.006]
sel_imp = sel_imp['column'].to_list()
df4 = df.loc[:,sel_imp]
y = df.loc[:,'進場訊號']
X = df4
X_res, Y_res = SMOTE(random_state = 42).fit_resample(X, y)
std = StandardScaler()
xgb = XGBClassifier(n_estimator = 1000,learning_rate = 0.1,max_depth = 9, min_child_weight = 5, subsample = 0.8, colsample_bytree = 0.8)
std_x = std.fit_transform(X_res)
X_train, X_test, y_train, y_test = train_test_split(std_x, Y_res, test_size = 0.2, random_state = 42, stratify = Y_res)
xgb.fit(X_train, y_train)
acc_train = accuracy_score(y_train, xgb.predict(X_train))
acc = accuracy_score(y_test, xgb.predict(X_test))
roc_train = roc_auc_score(y_train, xgb.predict(X_train))
roc_test = roc_auc_score(y_test, xgb.predict(X_test))
cf = confusion_matrix(y_test, xgb.predict(X_test))
imp2 = pd.DataFrame(columns =['column','importance'])
imp2['column'] = X.columns
imp2['importance'] = xgb.feature_importances_.round(4)
imp2 = imp2.sort_values(by = 'importance', ascending = False)
print('ACC_train:{0:.3}'.format(acc_train))
print('ACC_test:{0:.3}'.format(acc))
print('ROC_train{0:.3}'.format(roc_train))
print('ROC_test{0:.3}'.format(roc_test))
print(cf)
imp2.head(20)
#取重要性>0.02，欄位挑選完後為
xgb_col = ['季','稅前淨利成長率','不重分類至損益之項目－OCI','經常淨利成長率']

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


ACC_train:0.978
ACC_test:0.828
ROC_train0.978
ROC_test0.828
[[2406  337]
 [ 607 2135]]


,column,importance
0,季,0.1980
1,稅前淨利成長率,0.0511
2,不重分類至損益之項目－OCI,0.0250
81,經常淨利成長率,0.0200
3,常續淨利成長率,0.0143
16,稅後淨利成長率,0.0134
14,其他綜合損益－OCI,0.0123
12,合約負債－流動,0.0111
13,稅前盈餘變動率,0.0110
8,當期所得稅資產－流動,0.0107


In [ ]:
#未調參結果
y = df.loc[:,'進場訊號']
X = df[['季','稅前淨利成長率','不重分類至損益之項目－OCI','經常淨利成長率']]
X_res, Y_res = SMOTE(random_state = 42).fit_resample(X, y)
std = StandardScaler()
xgb = XGBClassifier()
std_x = std.fit_transform(X_res)
X_train, X_test, y_train, y_test = train_test_split(std_x, Y_res, test_size = 0.2, random_state = 42, stratify = Y_res)
xgb.fit(X_train, y_train)
acc_train = accuracy_score(y_train, xgb.predict(X_train))
acc = accuracy_score(y_test, xgb.predict(X_test))
roc_train = roc_auc_score(y_train, xgb.predict(X_train))
roc_test = roc_auc_score(y_test, xgb.predict(X_test))
cf = confusion_matrix(y_test, xgb.predict(X_test))
print('ACC_train:{0:.3}'.format(acc_train))
print('ACC_test:{0:.3}'.format(acc))
print('ROC_train{0:.3}'.format(roc_train))
print('ROC_test{0:.3}'.format(roc_test))
print(cf)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


ACC_train:0.745
ACC_test:0.742
ROC_train0.745
ROC_test0.742
[[2461  282]
 [1132 1610]]


In [ ]:
#Tuning max_depth, max_child_weight
parameter_1 = [{'max_depth':range(3,10,2), 'min_child_weight':range(1,6,2)}]
grid_1 = GridSearchCV(xgb,param_grid=parameter_1,cv=5)
grid_1.fit(X_train, y_train)
print(grid_1.best_params_)

In [ ]:
#Tuning gamma
parameter_2 = [{'gamma':[i/10.0 for i in range(0,5)]}]
grid_2 = GridSearchCV(XGBClassifier(max_depth=9),param_grid=parameter_2,cv=5)
grid_2.fit(X_train,y_train)
print(grid_2.best_params_)

In [ ]:
#tuning subsample, colsample
parameter_3 = [{'subsample':[i/10.0 for i in range(6,10)], 'colsample_bytree':[i/10.0 for i in range(6,10)]}]
grid_3 = GridSearchCV(XGBClassifier(max_depth=9,gamma = 0.3),param_grid=parameter_3,cv=5)
grid_3.fit(X_train,y_train)
print(grid_3.best_params_)

In [ ]:
#reg_alpha
parameter_4 = [{'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]}]
grid_4 = GridSearchCV(XGBClassifier(max_depth=9,gamma = 0.3, colsample_bytree=0.6,subsample=0.8),param_grid=parameter_4,cv=5)
grid_4.fit(X_train,y_train)
print(grid_4.best_params_)

In [ ]:
#Tuning learning_rate
parameter_5 = [{'learning_rate':[0.01,0.03,0.05,0.07,0.1,0.15,0.2]}]
grid_5 = GridSearchCV(XGBClassifier(max_depth=9,gamma = 0.3, colsample_bytree=0.6,subsample=0.8,reg_alpha=0.005),param_grid=parameter_5,cv=5)
grid_5.fit(X_train, y_train)
print(grid_5.best_params_)

In [ ]:
#XGB-BEST
y = df3.loc[:,'進場訊號']
X = df3[['季','稅前淨利成長率','不重分類至損益之項目－OCI','經常淨利成長率']]
std = StandardScaler()
xgb = XGBClassifier(max_depth=9,gamma = 0.3,colsample_bytree=0.6,subsample=0.8,reg_alpha=0.005,scale_pos_weight=21)
std_x = std.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(std_x, y, test_size = 0.2, random_state = 42, stratify = y)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
pred_train = xgb.predict(X_train)
acc_train = accuracy_score(y_train, xgb.predict(X_train))
acc = accuracy_score(y_test, xgb.predict(X_test))
roc_train = roc_auc_score(y_train, xgb.predict(X_train))
roc_test = roc_auc_score(y_test, xgb.predict(X_test))
cf = confusion_matrix(y_test, xgb.predict(X_test))
print('ACC_train:{0:.3}'.format(acc_train))
print('ACC_test:{0:.3}'.format(acc))
print('ROC_train{0:.3}'.format(roc_train))
print('ROC_test:{0:.3}'.format(roc_test))
print('Confusion Matrix:\n{}\n'.format(cf))
print('==================================================')
a = pd.DataFrame({'進場訊號':y_test,'預測進場訊號':pred})
b = pd.DataFrame({'進場訊號':y_train,'預測進場訊號':pred_train})
c = pd.concat([a,b],axis = 0)
d = df3[['代號','名稱','年份','季','開盤價','收盤價']]
e = pd.concat([d,c],axis = 1)
e = e.sort_values(by = ['代號','名稱','年份','季'])
final_2 = e
final_2.to_csv('./Data/Method_B_Final_XGB_Feature_Importances.csv', index=0, header=True, encoding='utf-8')


ACC_train:0.352
ACC_test:0.313
ROC_train0.563
ROC_test:0.523
Confusion Matrix:
[[ 239 2504]
 [  39  918]]



,代號,名稱,年份,季,開盤價,收盤價,進場訊號,預測進場訊號
18138,1471,首利,2010,1,34.20,23.75,0,1
17779,1471,首利,2010,2,23.85,24.50,0,1
17412,1471,首利,2010,3,24.50,20.10,0,1
17046,1471,首利,2010,4,20.20,17.15,0,1
16668,1471,首利,2011,1,17.15,13.90,0,1
...,...,...,...,...,...,...,...,...
2344,9912,偉聯,2019,4,8.44,5.80,0,1
1877,9912,偉聯,2020,1,5.76,6.82,1,1
1411,9912,偉聯,2020,2,6.77,7.54,1,1
943,9912,偉聯,2020,3,7.54,8.39,1,1


## 其他模型

In [ ]:
#AR_RF
y = df3.loc[:,'進場訊號']
X = df3[ar_col]
dim = 11
rf = RandomForestClassifier(n_estimators=1000,max_depth=30, max_features=3)
normal_training(X,y,dim,rf)

In [ ]:
#AR_SVM
y = df.loc[:,'進場訊號']
X = df[sel_col]
svc = SVC(C = 1000,gamma = 0.1, class_weight='balanced'})
dim = 4
normal_training(X, y, dim, svc)

In [ ]:
#XGB_SVM
y = df3.loc[:,'進場訊號']
X = df3[['季','稅前淨利成長率','不重分類至損益之項目－OCI','經常淨利成長率']]
std = StandardScaler()
svc = SVC(C=800, gamma=0.5, class_weight='balanced')
std_x = std.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(std_x, y, test_size = 0.2, random_state = 42, stratify = y)
svc.fit(X_train, y_train)
acc_train = accuracy_score(y_train, svc.predict(X_train))
acc = accuracy_score(y_test, svc.predict(X_test))
roc_train = roc_auc_score(y_train, svc.predict(X_train))
roc_test = roc_auc_score(y_test, svc.predict(X_test))
cf = confusion_matrix(y_test, svc.predict(X_test))
print('ACC_train:{0:.3}'.format(acc_train))
print('ACC_test:{0:.3}'.format(acc))
print('ROC_train{0:.3}'.format(roc_train))
print('ROC_test{0:.3}'.format(roc_test))
print(cf)


In [ ]:
#XGB_RF
y = df.loc[:,'進場訊號']
X = df[['季','稅前淨利成長率','不重分類至損益之項目－OCI','經常淨利成長率']]
X_res, Y_res = SMOTE(random_state = 42).fit_resample(X, y)
std = StandardScaler()
rf = RandomForestClassifier(n_estimators = 500, max_depth = 50, max_features=2, min_samples_split=2)
std_x = std.fit_transform(X_res)
X_train, X_test, y_train, y_test = train_test_split(std_x, Y_res, test_size = 0.2, random_state = 42, stratify = Y_res)
rf.fit(X_train, y_train)
acc_train = accuracy_score(y_train, rf.predict(X_train))
acc = accuracy_score(y_test, rf.predict(X_test))
roc_train = roc_auc_score(y_train, rf.predict(X_train))
roc_test = roc_auc_score(y_test, rf.predict(X_test))
cf = confusion_matrix(y_test, rf.predict(X_test))
print('ACC_train:{0:.3}'.format(acc_train))
print('ACC_test:{0:.3}'.format(acc))
print('ROC_train:{0:.3}'.format(roc_train))
print('ROC_test:{0:.3}'.format(roc_test))
print(cf)

In [ ]:
#PCA_RF
y = df['進場訊號']
X = df.loc[:,'季':'存貨週轉率（次）']
rf = RandomForestClassifier(max_depth =20, min_samples_split=3)
dim = 100
normal_training(X,y,dim,rf)


In [ ]:
#PCA_SVM
y = df['進場訊號']
X = df.loc[:,'季':'存貨週轉率（次）']
svc = SVC(C = 100, gamma = 0.1, class_weight='balanced')
dim = 110
normal_training(X,y,dim,svc)